In [2]:
import numpy as np
import pandas as pd
import re
import os
import glob


In [3]:
print(os.getcwd())


/home/smmrrr/TEM_Analysis


In [7]:
# get processed files

paths = glob.glob(
    '*merged_dataset*GL*'
)
i = 1
len(paths)


40

In [5]:
##read in global, annual CO2

co2 = pd.read_csv('/group/moniergrp/TEM/climate/IGSM-CAM_windDrnd10_CS2p5_Fae-0p55_Kz0p05_bau/co2.1869-2100.glb'
               ,  sep = '\s+'
                 , names = ['co2'])


co2 = co2.reset_index()
co2 = co2.rename(columns = {'index':'year'})



In [6]:
for i in range(1, len(paths)):
    merged_dataset = pd.read_csv(paths[i])

    cohort_area = lambda x: np.average(x, weights=merged_dataset.loc[x.index, "cohort_area"])  # https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe
    land_area = lambda x: np.average(x, weights=merged_dataset.loc[x.index, "land_area"])  # https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe

    if (i==1):


        merged_dataset_annual_loop = merged_dataset.groupby([
                'year','current_veg', 'stand_age']).agg(cohort_area=('cohort_area', 'sum'), total_area=('land_area', 'sum'),
                                                                                      NCE_weighted=('NCE', cohort_area), 
                                                                                   prec = ('prec', land_area),
                                                                                   aot40 = ('aot40', land_area),
                                                                                   tair = ('tair', land_area),
                                                                                   nirr = ('nirr', land_area))
    else:
                                                        
        merged_dataset_annual_loop = pd.concat([merged_dataset_annual_loop,
                              merged_dataset.groupby([
                'year','current_veg', 'stand_age']).agg(cohort_area=('cohort_area', 'sum'), total_area=('land_area', 'sum'),
                                                                                      NCE_weighted=('NCE', cohort_area), 
                                                                                   prec = ('prec', land_area),
                                                                                   aot40 = ('aot40', land_area),
                                                                                   tair = ('tair', land_area),
                                                                                   nirr = ('nirr', land_area))])
                                                                                    

In [ ]:
merged_dataset_annual_loop = merged_dataset_annual_loop.reset_index()
merged_dataset_annual_loop = merged_dataset_annual_loop.merge(co2, how = 'left', on = 'year')

In [ ]:
merged_dataset_annual_loop.to_csv('merged_dataset_annual_loop.csv')